<center><img src="images/DLI Header.png" alt="Header" width="400"></center>

# 5.0  Video File Output
It may be desirable to save an annotated file for later playback and analysis. 

Just as there are pipeline elements that decode and parse a file read from `filesrc` into a stream, the pattern can be reversed to save a video to file using `filesink`.  In this notebook, you'll learn how to add the elements required to encode, parse, multiplex, and save a video to file.

<img src="images/04_example_mp4_out.png" alt="pipe to file">

**[5.1 Build a Pipeline that Outputs to a File](#5.1-Build-a-Pipeline-that-Outputs-to-a-File)**<br>
&nbsp; &nbsp; &nbsp;[5.1.1 Practice Application `deepstream-test3-mp4-out`](#5.1.1-Practice-Application-deepstream-test3-mp4-out)<br>
&nbsp; &nbsp; &nbsp;[5.1.2 Exercise: Run the Base Application](#5.1.2-Exercise:-Run-the-Base-Application)<br>
**[5.2 Create an Annotated Video File](#5.2-Create-an-Annotated-Video-File)**<br>
&nbsp; &nbsp; &nbsp;[5.2.1 Exercise: Output Annotated Video to File](#5.2.1-Exercise:-Output-Annotated-Video-to-File)<br>
**[5.3 Put It All Together](#5.3-Put-It-All-Together)**<br>
&nbsp; &nbsp; &nbsp;[5.3.1 Exercise: Change the File Type](#5.3.1-Exercise:-Change-the-File-Type)<br>

# 5.1 Build a Pipeline that Outputs to a File
In all of the RTSP output apps used so far in this course, the pipeline ended with the following sequence of plugin elements:
- `Gst-nvv4l2h264enc` - encodes RAW data in I420 format to H264
- `GstRtpH264Pay` - converts H264 encoded Payload to RTP packets (RFC 3984)
- `GstUDPSink` - sends UDP packets to the network. When paired with RTP payloader (`Gst-rtph264pay`) it can implement RTP streaming

To save a video stream to a file, we still use the H264 encoding plugin, `Gst-nvv4l2h264enc`, but not the `GstRtp264Pay` plugin for RTSP, nor the `GstUDPSink` plugin.  This is because the stream won't be transmitted over the network.  Instead, we need to **parse** the encoded stream and **multiplex** (sometimes called "mux") it into a convenient media container format, such as an ISO MPEG-4 `.mp4` file.  The `filesink` element becomes the new sink.<br>
In summary, saving our rendered streams to a file requires the pipeline after rendering to be:
- `Gst-nvv4l2h264enc` - encodes RAW data in I420 format to H264
- `GstH264Parse` - parses the encoded H264 stream
- `GstMP4Mux` -  merges streams (audio and video) into ISO MPEG-4 (.mp4) files.
- `GstFileSink` - writes incoming data to a file in the local file system

## 5.1.1 Practice Application `deepstream-test3-mp4-out`
A sample app, based on the `deepstream-test3` reference app, sends the output to a file that can be viewed.

In [ ]:
# Set some path locations for readability
PYTHON_APPS = '/opt/nvidia/deepstream/deepstream/sources/deepstream_python_apps/apps'
STREAMS = '/opt/nvidia/deepstream/deepstream/samples/streams'
DLI_APPS = '/opt/nvidia/deepstream/deepstream/sources/deepstream_python_apps/dli_apps'
MY_APPS = '/opt/nvidia/deepstream/deepstream/sources/deepstream_python_apps/my_apps'
# Make sure common utilities are in my_apps
!cp -r $PYTHON_APPS/common $MY_APPS/

# List the contents of the sample app
!ls $DLI_APPS/deepstream-test3-mp4-out

A comparison of the [deepstream-test3-rstp-out/deepstream_test3_rtsp_out.py](deepstream/sources/deepstream_python_apps/dli_apps/deepstream-test3-rtsp-out/deepstream_test3_rtsp_out.py) and the [deepstream-test3-mp4-out/deepstream_test3_mp4_out.py](deepstream/sources/deepstream_python_apps/dli_apps/deepstream-test3-mp4-out/deepstream_test3_mp4_out.py), the equivalent application that writes to file, reveals the following differences *(Note: the sample snippets below are abbreviated for clarity)*:

RTSP streaming `deepstream-test3-rstp-out` code in `main()`:
```python
...

    # Make the payload-encode video into RTP packets
    if codec == "H264":
        rtppay = Gst.ElementFactory.make("rtph264pay", "rtppay")
        print("Creating H264 rtppay")
    elif codec == "H265":
        rtppay = Gst.ElementFactory.make("rtph265pay", "rtppay")
        print("Creating H265 rtppay")
    if not rtppay:
        sys.stderr.write(" Unable to create rtppay")
    
    # Make the UDP sink
    updsink_port_num = 5400
    sink = Gst.ElementFactory.make("udpsink", "udpsink")
    if not sink:
        sys.stderr.write(" Unable to create udpsink")
    
    sink.set_property('host', '224.224.255.255')
    sink.set_property('port', updsink_port_num)
    sink.set_property('async', False)
    sink.set_property('sync', 1)
    
    print("Playing file %s " %stream_path)
    source.set_property('location', stream_path)

...

    print("Adding elements to Pipeline \n")
    pipeline.add(pgie)
    pipeline.add(tiler)
    pipeline.add(nvvidconv)
    pipeline.add(nvosd)
    
#####################  RTSP
    pipeline.add(nvvidconv_postosd)
    pipeline.add(caps)
    pipeline.add(encoder)
    pipeline.add(rtppay)
    pipeline.add(sink)
#####################

...
    
    print("Linking elements in the Pipeline \n")
    streammux.link(queue1)
    queue1.link(pgie)
    pgie.link(queue2)
    queue2.link(tiler)
    tiler.link(queue3)
    queue3.link(nvvidconv)
    nvvidconv.link(queue4)
    queue4.link(nvosd)   
        
#####################  RTSP
    nvosd.link(queue5)
    queue5.link(nvvidconv_postosd)
    nvvidconv_postosd.link(caps)
    caps.link(encoder)
    encoder.link(rtppay)
    rtppay.link(sink)
#####################

...
```

File ouptut `deepstream-test3-mp4_out` code in `main()`:
```python
...
    
# added for mp4-out
    codecparse = Gst.ElementFactory.make("h264parse", "h264_parse")
    if not codecparse:
        sys.stderr.write(" Unable to create codecparse \n")
        
    mux = Gst.ElementFactory.make("mp4mux", "mux")
    if not mux:
        sys.stderr.write(" Unable to create mux \n")

    sink = Gst.ElementFactory.make("filesink", "filesink")
    if not sink:
        sys.stderr.write(" Unable to create filesink \n")
    sink.set_property('location', output_path)

...
    
    print("Adding elements to Pipeline \n")
    pipeline.add(pgie)
    pipeline.add(tiler)
    pipeline.add(nvvidconv)
    pipeline.add(nvosd)
    
#####################  RTSP
    pipeline.add(nvvidconv_postosd)
    pipeline.add(caps)
    pipeline.add(encoder)
    pipeline.add(codecparse)
    pipeline.add(mux)
#     pipeline.add(rtppay)
    pipeline.add(sink)

...
    
    print("Linking elements in the Pipeline \n")
    streammux.link(queue1)
    queue1.link(pgie)
    pgie.link(queue2)
    queue2.link(tiler)
    tiler.link(queue3)
    queue3.link(nvvidconv)
    nvvidconv.link(queue4)
    queue4.link(nvosd)
        
#####################  RTSP
    nvosd.link(queue5)
    queue5.link(nvvidconv_postosd)
    nvvidconv_postosd.link(caps)
    caps.link(encoder)
    encoder.link(codecparse)
    codecparse.link(mux)
    mux.link(sink)
#     encoder.link(rtppay)
#     rtppay.link(sink)
#####################

...
```

Note the basic differences:
* `rtppay` has been removed (`rtph264pay` element) 
* `codecparse` and `mux` have been added (`h264parse` and `mp4mux` elements)
* `sink` has been changed (from a `udspsink` element to a `filesink` element)
   * no `host`, `port`, `sync`, `async` properties used
   * `location` property is required for `filesink`

To parameterize the output file name (the variable `output_path`), an additional option is added to the `parse_args()` definition with the following lines of code:
```python
    parser.add_argument("-o", "--output", default='/dli/task/out.mp4',
                  help="Set the output file path ")
    global output_path
    output_path = args.output
```

## 5.1.2 Exercise: Run the Base Application

In [ ]:
# Check usage of the test3 mp4 app with the help option
!cd $DLI_APPS/deepstream-test3-mp4-out \
    && python3 deepstream_test3_mp4_out.py --help

In [ ]:
# Run the app with two input streams
OUTPUT_PATH_EX1 = '/dli/task/nb5_out.mp4'
!cd $DLI_APPS/deepstream-test3-mp4-out \
    && python3 deepstream_test3_mp4_out.py -o $OUTPUT_PATH_EX1 -i \
        file://$STREAMS/sample_720p.h264 \
        file://$STREAMS/sample_720p.mp4

In [ ]:
# Watch the saved video
import os
from IPython.display import Video

video_path = os.path.relpath(OUTPUT_PATH_EX1)
Video(video_path, width = 600, height = 300)

#### How did you do?
If you see something like this image when you play your saved file, you did it!  If not, keep trying, or take a peek at the solution code in the solutions directory.  <br>

<img src="images/04_test3_mp4_out.png" alt="file output test1">

# 5.2 Create an Annotated Video File
All of the RSTP samples you've worked with so far include the same elements after rendering.  You can modify any of them to output to file in the same way as `deepstream-test3-mp4-out`.

## 5.2.1 Exercise: Output Annotated Video to File
Create a new app based on `deepstream-test1-rtsp-out` that saves the annotated output to a file at `/dli/task/nb5_test1_out.mp4`.

In [ ]:
# Create a new app located at $MY_APPS/dst1-mp4-out 
#      based on $PYTHON_APPS/deepstream-test1-rtsp-out
!mkdir -p $MY_APPS/dst1-mp4-out
!cp -rfv $PYTHON_APPS/deepstream-test1-rtsp-out/* $MY_APPS/dst1-mp4-out
# Rename your Python file for clarity
!mv $MY_APPS/dst1-mp4-out/deepstream_test1_rtsp_out.py \
    $MY_APPS/dst1-mp4-out/deepstream_test1_mp4_out.py

Modify your [deepstream_test1_mp4_out.py](my_apps/dst1-mp4-out/deepstream_test1_mp4_out.py) Python file with the following steps (refer to the code differences highlighted in the previous section for details):
1. Assign `codecparse` and `mux` as GstElement objects
2. Remove the `rtph264pay` element.
3. Add `h264parse` and `mp4mux` elements (assigned to `codecparse` and `mux`).
4. Change `sink` from a `udspsink` element to a `filesink` element.
   - Remove the "host" property that is no longer applicable.
   - Set the "location" property.
5. Change the `pipeline.add` and `.link` as needed to reflect the new pipeline.
6. Adjust the `parse_args()` definition to include the output file parameter.  Use the definition `parse_args()` definition provided in [deepstream_test3_mp4_out.py](deepstream/sources/deepstream_python_apps/dli_apps/deepstream-test3-mp4-out/deepstream_test3_mp4_out.py) as a guide.

      ```python
   # hint - add these three lines
      parser.add_argument("-o", "--output", default='/dli/task/out.mp4',
              help="Set the output file path ")
      global output_path
      output_path = args.output
      ```

In [ ]:
# Check usage of your new test1 mp4 app with the help option
!cd $MY_APPS/dst1-mp4-out \
    && python3 deepstream_test1_mp4_out.py --help

In [ ]:
# Run the app
OUTPUT_PATH_EX2 = '/dli/task/nb5_test1_out.mp4'
!cd $MY_APPS/dst1-mp4-out \
    && python3 deepstream_test1_mp4_out.py \
        -o $OUTPUT_PATH_EX2 \
        -i $STREAMS/sample_720p.h264

In [ ]:
# Watch the saved video
import os
from IPython.display import Video

video_path = os.path.relpath(OUTPUT_PATH_EX2)
Video(video_path, width = 600, height = 300)

#### How did you do?
If you see something like this image when you play your saved file, you did it!  If not, keep trying, or take a peek at the [solution code](solutions/ex5.2.1_test1_to_file/dst1-mp4-out/deepstream_test1_mp4_out.py) in the solutions directory.  <br>

<img src="images/04_test1_mp4_out.png" alt="file output test1">

# 5.3 Put It All Together
Try it again with a few small enhancements, starting with the recommended app in the instructions, or one of your own from earlier in the course.  

## 5.3.1 Exercise: Change the File Type
Add the following enhancements:
1. Replace the RTSP portion of the pipeline with file output elements.
1. Instead of `mp4mux`, which saves to `.mp4` format, try the `avimux` element and save to the `.avi` format.  
1. Alter the `parse_args()` definition to include an output file name option (as with the previous exercise).
1. Save your output file to a file at `/dli/task/nb5_test2_out.avi`

In [ ]:
OUTPUT_PATH_EX3 = '/dli/task/nb5_test2_out.avi'

In [ ]:
# TODO
# Create a new app located at $MY_APPS/dst2-avi-out 
#      based on $PYTHON_APPS/deepstream-test2-rtsp-out

In [ ]:
#TODO
# Check usage of your new test2 avi app with the help option

In [ ]:
# TODO
# Run the app

#### Watch the video on your computer
The `.avi` format cannot be viewed in this notebook.  To download the file to your computer, right-click from the file browser and select "download".  You can use the VLC Media Player ("Media"->"Open File") to play the video on your computer.

#### How did you do?
If you see something like this image when you play your saved file, you've mastered saving your annotated videos to a file!  If not, keep trying, or take a peek at the [solution code](solutions/ex5.3.1_test2_to_file/solution-5.3.1.ipynb) in the solutions directory.  <br>

<img src="images/02_3sgie.png" alt="file output test2">

<h2 style="color:green;">Congratulations!</h2>

You modified a streaming DeepStream pipeline to instead save a video to a file.<br>
Move on to [6.0 Different Neural Networks](./06_DiffNetworks.ipynb)


<center><img src="images/DLI Header.png" alt="Header" width="400"></center>